In [1]:
import pandas as pd
import numpy as np

### Cargar datos

In [2]:
listings = pd.read_csv("Beijing Airbnb Data(clean)/listings_clean.csv")

In [3]:
calendar = pd.read_csv("Beijing Airbnb Data(clean)/calendar_detail.csv")

In [4]:
reviews = pd.read_csv("Beijing Airbnb Data(clean)/reviews_detail.csv")

In [28]:
print(listings.shape,reviews.shape,calendar.shape)

(28448, 10) (21425, 5) (10384980, 7)


#### Pre-procesamiento listings

In [9]:
listings['last_review']=listings['last_review'].replace(np.nan, 'No reviews', regex=True)
listings=listings.drop(['neighbourhood','number_of_reviews','last_review','reviews_per_month',
                        'calculated_host_listings_count'], axis=1)

In [10]:
listings.isna().sum()

id                  0
name                0
host_id             0
host_name           0
latitude            0
longitude           0
room_type           0
price               0
minimum_nights      0
availability_365    0
dtype: int64

In [11]:
listings.to_csv('listings_final.csv',index=False)

In [21]:
listings.id.array

<PandasArray>
[   44054,   100213,   128496,   161902,   162144,   279078,   282825,
   287026,   287511,   314453,
 ...
 33947515, 33947519, 33947559, 33948143, 33948296, 33948728, 33948787,
 33950006, 33950535, 33954414]
Length: 28448, dtype: int64

#### Pre-procesamiento reviews

In [6]:
reviews=reviews[reviews.comments.str.contains(r'[^\x00-\x7F]+')== False]
reviews=reviews[reviews.reviewer_name.str.contains(r'[^\x00-\x7F]+')== False]
reviews=reviews.drop('reviewer_id', axis=1)

In [25]:
for x in reviews.listing_id.array:
    if x not in listings.id.array:
        reviews.drop(reviews.loc[reviews['listing_id']==x].index, inplace=True)

In [26]:
reviews

,listing_id,id,date,reviewer_name,comments
0,44054,84748,2010-08-25,Jarrod,Sev was very helpful. Sev showed us where to ...
1,44054,118384,2010-10-13,Kimberly,We arrived in Beijing very early in the mornin...
2,44054,436978,2011-08-11,Emma,It is a really massive apartment and really co...
3,44054,1118657,2012-04-12,Andreyna,"Sev was incredibly helpful, showed us around t..."
4,44054,2140650,2012-08-30,Frances,The appartment was ideal for our party of 6 ad...
...,...,...,...,...,...
201976,33687740,436829448,2019-04-14,Chris & Loretta,"Responds to questions were quickly answered, g..."
201995,33703447,437685367,2019-04-15,Wei,The host canceled this reservation the day bef...
202019,33756022,438645023,2019-04-17,Ying,The host canceled this reservation 12 days bef...
202023,33757621,436102028,2019-04-12,Ervinas,The host canceled this reservation the day bef...


In [27]:
reviews.to_csv('reviews_final.csv',index=False)

#### Pre-procesamiento calendar

In [12]:
sample_calendar = calendar.groupby('listing_id').apply(lambda x: x.sample(frac=0.005))
sample_calendar['price']=sample_calendar['price'].str.replace(r',', '')
sample_calendar['price']=sample_calendar['price'].str.replace(r'$', '')
sample_calendar['adjusted_price']=sample_calendar['adjusted_price'].str.replace(r',', '')
sample_calendar['adjusted_price']=sample_calendar['adjusted_price'].str.replace(r'$', '')

/home/cris/miniconda3/envs/PLN/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/cris/miniconda3/envs/PLN/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """


In [13]:
sample_calendar.reset_index(drop=True, inplace=True)
sample_calendar.dropna(inplace=True)

In [14]:
sample_calendar.head(100)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,44054,2019-05-23,f,792.00,792.00,1.0,365.0
1,44054,2019-05-31,f,792.00,792.00,1.0,365.0
2,100213,2019-09-20,f,1320.00,1320.00,1.0,30.0
3,100213,2020-04-12,f,1200.00,1200.00,1.0,30.0
4,128496,2019-12-20,f,389.00,389.00,3.0,365.0
...,...,...,...,...,...,...,...
95,1303170,2020-03-29,t,1155.00,1155.00,1.0,60.0
96,1325945,2019-10-22,t,298.00,298.00,2.0,1125.0
97,1325945,2020-03-14,t,298.00,298.00,2.0,1125.0
98,1370541,2020-03-12,f,75.00,75.00,3.0,30.0


In [15]:
sample_calendar.head(100)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,44054,2019-05-23,f,792.00,792.00,1.0,365.0
1,44054,2019-05-31,f,792.00,792.00,1.0,365.0
2,100213,2019-09-20,f,1320.00,1320.00,1.0,30.0
3,100213,2020-04-12,f,1200.00,1200.00,1.0,30.0
4,128496,2019-12-20,f,389.00,389.00,3.0,365.0
...,...,...,...,...,...,...,...
95,1303170,2020-03-29,t,1155.00,1155.00,1.0,60.0
96,1325945,2019-10-22,t,298.00,298.00,2.0,1125.0
97,1325945,2020-03-14,t,298.00,298.00,2.0,1125.0
98,1370541,2020-03-12,f,75.00,75.00,3.0,30.0


In [16]:
sample_calendar.isna().sum()

listing_id        0
date              0
available         0
price             0
adjusted_price    0
minimum_nights    0
maximum_nights    0
dtype: int64

In [17]:
sample_calendar.to_csv('calendar_final.csv',index=False)